# Exp 02: Try using CuPy with the matrices
Can we find parent/child word relationships faster  
One way to achieve this is to use the GPU via CuPy 
https://docs.cupy.dev/en/stable/index.html  
CuPy is basically NumPy on the GPU. Let's start with a simple case.  

In [1]:
# standard libraries
from time import perf_counter_ns
import time

In [2]:
# external libraries
import pandas as pd
import numpy as np

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *

# Load input data
The char_matrix and the formatted words

In [3]:
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)

...loading words into a dataframe...
...query execution took: 0.51 seconds...
...loading word groups into a dataframe...
...query execution took: 0.5 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


# let's process 1000 rows using a single lookup on the full wchar_matrix

In [4]:
# load the total number of anagrams
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH,
                                             db_name=rc.DB_NAME)

...query execution took: 0.0 seconds...


In [5]:
f"{n_possible_anagrams :,}"

'99,421,122'

# Using the CPU
Create a data extraction scenario that is very similar to Matrix Extraction Technique 1: Using the full matrix. I remove the excess code and create a simple loop. 

In [6]:
run_start_time = perf_counter_ns()
# sample 1000 rows
n_samples = 1000
sample_wg_id = wg_df['word_group_id'].sample(n = n_samples, random_state = 42).to_list()

# let's add a specific word: acanthology - the study of spines (as of sea urchins) especially as an adjunct of taxonomy
# this is to ensure that extraction technique produces the correct values
# On a complete run, there should by 26 parent words and 329 child words
# This will find the 26 parent words, and depending on what else in the sample,
# a number of child words

if 746 not in sample_wg_id:
    sample_wg_id.append(746)    

# create an output object
output_list = np.full(shape=(n_possible_anagrams, 2),
                          fill_value=-1, dtype=np.int32)

anagram_pair_count = 0

for i_wg_id, wg_id in enumerate(sample_wg_id):
    # identify parent words
    outcome = wchar_matrix - wchar_matrix[wg_id, ]
    
    # compute the score by finding where rows, across all columns, are GTE 0
    outcome_indices = np.all(outcome >= 0, axis=1)    

    n_from_words = outcome_indices.sum()

    if n_from_words >= 1:

        outcome_word_id_list = word_group_id_list[outcome_indices]
        # extract anagrams based on index values    
        outcome_word_id_list = format_output_list(outcome_word_id_list=outcome_word_id_list,
                                                  wg_id=wg_id)

        # enumerate the from/parent words
        new_anagram_pair_count = anagram_pair_count + n_from_words

        output_list[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list
        
        # set the anagram pair count
        anagram_pair_count = new_anagram_pair_count    
    
    if i_wg_id % 100 == 0:
        print(i_wg_id)
        
print('...time to find parent words...')
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)
print('...truncating output list...')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
print(output_list.shape)
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)

0
100
200
300
400
500
600
700
800
900
1000
...time to find parent words...
Hours: 0 | minutes: 0 | seconds: 6.5234
...truncating output list...
(367672, 2)
Hours: 0 | minutes: 0 | seconds: 7.2889


In [7]:
from_word_counter, to_word_counter = build_counters(output_list=output_list)

In [8]:
# the number of from word groups: should be 26
from_word_counter[746]

np.int64(26)

# Setup a similar workflow, using CuPy

In [9]:
# load the library
import cupy as cp

In [10]:
# perform a simple benchmark
from cupyx.profiler import benchmark

def my_func(a):
    return cp.sqrt(cp.sum(a**2, axis=-1))

a = cp.random.random((256, 1024))
print(benchmark(my_func, (a,), n_repeat=20))  


my_func             :    CPU:    63.820 us   +/- 17.857 (min:    48.200 / max:   127.700) us     GPU-0:   203.749 us   +/- 16.897 (min:   191.488 / max:   267.072) us


In [11]:
# initialize some gpu stats
# this page, https://docs.cupy.dev/en/stable/user_guide/performance.html,
# talks about performance best practices
start_gpu = cp.cuda.Event()
end_gpu = cp.cuda.Event()

start_gpu.record()
start_cpu = time.perf_counter()
out = my_func(a)
end_cpu = time.perf_counter()
end_gpu.record()
end_gpu.synchronize()
t_gpu = cp.cuda.get_elapsed_time(start_gpu, end_gpu)
t_cpu = end_cpu - start_cpu
print(t_gpu, t_cpu)

0.7086079716682434 0.000685499999917738


## Create cupy objects

In [12]:
char_matrix_cp = cp.asarray(a = char_matrix, dtype = cp.int8)
word_group_id_list_cp = cp.asarray(a = word_group_id_list, dtype = cp.int32)
word_id_list = cp.asarray(a = word_id_list, dtype = cp.int32)
wchar_matrix_cp = cp.asarray(a = wchar_matrix, dtype = cp.int8)
word_group_id_list_cp = cp.asarray(a = word_group_id_list, dtype = cp.int32)


In [13]:
def format_output_list_cp(outcome_word_id_list_cp: cp.ndarray, wg_id: int) -> cp.ndarray:
    output_list = cp.zeros(
        shape=(outcome_word_id_list_cp.shape[0], 2), dtype=cp.int32)

    # update the output list with the word_id_list - these are from/parent words
    output_list[:, 0] = outcome_word_id_list_cp

    # update with the word_id - this is the to/child word
    output_list[:, 1] = wg_id

    return output_list


In [14]:
run_start_time = perf_counter_ns()
# the naive approach
# establish counters for record keeping
# sample 1000 rows
n_samples = 1000

#sample_wg_id = wg_df['word_group_id'].sample(n = n_samples, random_state = 42)
sample_wg_id = wg_df['word_group_id'].to_numpy(dtype = np.int32)
sample_wg_id_cp = cp.asarray(a = sample_wg_id, dtype = cp.int32)

output_list_cp = cp.full(shape=(n_possible_anagrams, 2),
                          fill_value=-1, dtype = cp.int32)

row_count = 0
anagram_pair_count = 0
intermediate_to_word_count = collections.Counter()

for wg_id in sample_wg_id_cp:
    outcome = wchar_matrix_cp - wchar_matrix_cp[wg_id, ]
    
    # compute the score by finding where rows, across all columns, are GTE 0
    outcome_indices_cp = cp.all(outcome >= 0, axis=1)
    outcome = None

    n_from_words = outcome_indices_cp.sum()               
    
    if n_from_words >= 1:
        # extract anagrams based on index values    
        outcome_word_id_list_cp = word_group_id_list_cp[outcome_indices_cp]    
        
        # extract anagrams based on index values    
        outcome_word_id_list_cp = format_output_list_cp(outcome_word_id_list_cp=outcome_word_id_list_cp,
                                                  wg_id=wg_id)
        
        # enumerate the from/parent words
        new_anagram_pair_count = anagram_pair_count + n_from_words

        # update the total output list
        output_list_cp[anagram_pair_count:new_anagram_pair_count,
                    :] = outcome_word_id_list_cp

        # set the anagram pair count
        anagram_pair_count = new_anagram_pair_count
    
    row_count += 1

    if row_count % 10000 == 0:
        print(row_count)

print('...time to find parent words...')
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)
print('...truncating output list...')
output_indices = cp.all(output_list_cp >= 0, axis=1)
output_list_cp = output_list_cp[output_indices,]
print(output_list_cp.shape)
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
...time to find parent words...
Hours: 0 | minutes: 2 | seconds: 41.6578
...truncating output list...
(73218235, 2)
Hours: 0 | minutes: 2 | seconds: 41.8155


In [15]:
# very cool! this took about 1 second on a sample of 1000 words
# as expected, the GPU is faster.
# but, what about a full run?
# on all ~216K words, it takes a little less than 3 minutes.

# Implement a variant of matrix extraction 5: using the three least common letters
The variant here is that letter selectors are not stored - they are used once. With a max of three letters in each selector, there are 2,387 letter selectors. Each sub-matrix is created, words are queried against it, and the values are stored. This implementation, while similar to the existing version of matrix extraction technique 5, is a little faster than the existing version.

['Exp_01_demo_numpy_data_types.ipynb'](Exp_01_demo_numpy_data_types.ipynb) Features the implementation for the CPU. Below is just for the GPU.

In [16]:
# let's split the matrix, gather the values for each split, and then combine
n_subset_letters = 3


In [17]:
wg_df['n_records'] = int(1)

In [18]:
ls_df = build_letter_selector_df(df = wg_df,
                          ls_nchar = n_subset_letters,
                          letter_selector_col_name = 'letter_selector',                          
                          letter_selector_id_col_name = 'letter_selector_id')

In [19]:
ls_df.shape

(2387, 5)

In [20]:
ls_df.head()

,letter_selector,ls_count,ls_nchar_iter,ls_nchar,letter_selector_id
0,a,2,3,1,0
1,ae,1,3,2,1
2,ai,1,3,2,2
3,b,1,3,1,3
4,ba,4,3,2,4


In [21]:
ls_df['ls_count'].sum()

np.int64(215842)

In [22]:
ls_df['ls_count'].describe()

count    2387.000000
mean       90.423963
std       212.422095
min         1.000000
25%         3.000000
50%        14.000000
75%        75.000000
max      2544.000000
Name: ls_count, dtype: float64

In [23]:
ls_df = get_ls_index(df = ls_df)

...loading the letter dictionary...


In [24]:
ls_df.head()

,letter_selector,ls_count,ls_nchar_iter,ls_nchar,letter_selector_id,ls_index
0,a,2,3,1,0,"[True, False, False, False, False, False, Fals..."
1,ae,1,3,2,1,"[True, False, False, False, True, False, False..."
2,ai,1,3,2,2,"[True, False, False, False, False, False, Fals..."
3,b,1,3,1,3,"[False, True, False, False, False, False, Fals..."
4,ba,4,3,2,4,"[True, True, False, False, False, False, False..."


In [25]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,n_records,letter_selector
0,A,a,1,a,0,0,a,a,1,1,a
1,aa,aa,2,a,1,1,a,a,1,1,a
2,aal,aal,3,a,2,2,al,la,2,1,la
3,aalii,aalii,5,a,3,3,ail,lai,1,1,lai
4,aam,aam,3,a,4,4,am,ma,2,1,ma


In [26]:
col_names = ['letter_selector', 'letter_selector_id']
wg_df = pd.merge(left = wg_df, right = ls_df[col_names])

In [27]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,n_records,letter_selector,letter_selector_id
0,A,a,1,a,0,0,a,a,1,1,a,0
1,aa,aa,2,a,1,1,a,a,1,1,a,0
2,aal,aal,3,a,2,2,al,la,2,1,la,1081
3,aalii,aalii,5,a,3,3,ail,lai,1,1,lai,1083
4,aam,aam,3,a,4,4,am,ma,2,1,ma,1114


In [28]:
# let's create another matrix that features the letter_selector_id and the corresponding id
# of each word_group
ls_id_wg_id, ls_index_array = build_ls_index_arrays(wg_df=wg_df, ls_df=ls_df)

# Do it with CUPY

In [29]:
def format_output_list_cp(outcome_word_id_list_cp: cp.ndarray, wg_id: int) -> cp.ndarray:
        
    output_list_cp = cp.zeros(shape=(outcome_word_id_list_cp.shape[0], 2), dtype=cp.int32)

    # update the output list with the word_id_list - these are from/parent words
    output_list_cp[:, 0] = outcome_word_id_list_cp

    # update with the word_id - this is the to/child word
    output_list_cp[:, 1] = wg_id

    return output_list_cp

In [30]:
# create some more cupy objects
ls_index_array_cp = cp.asarray(ls_index_array)
ls_id_wg_id_cp = cp.asarray(ls_id_wg_id, dtype = cp.int32)  


In [31]:
run_start_time = perf_counter_ns()
# create the output list
output_list_cp = cp.full(shape = (n_possible_anagrams, 2), fill_value=-1, dtype = cp.int32)

output_time_list = []

# start counting
anagram_pair_count = 0

for ls_row_id, ls_row in enumerate(ls_index_array_cp):        
    if ls_row_id % 100 == 0:
        print(ls_row_id)
    start_time = perf_counter_ns()
        
    ##
    # SUBSET THE wchar_matrix by column selector
    ##        
    outcome_indices_cp = cp.all(wchar_matrix_cp[:, ls_row] >= 1, axis=1)    
    
    # this is the sub-matrix from which to query
    ls_wchar_matrix_cp = wchar_matrix_cp[outcome_indices_cp, :]
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    temp_wg_id_list_cp = word_group_id_list_cp[outcome_indices_cp]
    # place into a dictionary to go from wg_id to wg_index.
    # What is the index of wg_id 675?    

    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'buc'    
    n_search_space = temp_wg_id_list_cp.shape[0]        
       
    # the current list of words featuring the set of least common letters.
    # these are the words have the least common letters of 'buc'    
    curr_wg_id_list_cp = ls_id_wg_id_cp[ls_id_wg_id_cp[:, 0] == ls_row_id, 1]    
    
    # enumerate each word in the word group
    for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list_cp):    
        
        temp_wg_id = cp.where(temp_wg_id_list_cp == curr_wg_id)[0][0]        
        
        outcome_word_id_list_cp = temp_wg_id_list_cp[cp.all(a = (ls_wchar_matrix_cp - ls_wchar_matrix_cp[temp_wg_id, :]) >= 0, axis = 1)]
                
        n_from_words = outcome_word_id_list_cp.shape[0]
        
        if n_from_words > 0:
            outcome_word_id_list_cp = format_output_list_cp(outcome_word_id_list_cp=outcome_word_id_list_cp, wg_id=curr_wg_id)                       
            
            new_anagram_pair_count = anagram_pair_count + n_from_words            

            output_list_cp[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list_cp

            # update the anagram pair count
            anagram_pair_count = new_anagram_pair_count

    curr_time = calc_time(time_start=start_time, round_digits=8)
    output_time_list.append([ls_row_id, n_search_space, curr_time])

print('...time to find parent words...')
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)
print('...truncating output list...')
output_indices = cp.all(output_list_cp >= 0, axis=1)
output_list_cp = output_list_cp[output_indices,]
print(output_list_cp.shape)
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
...time to find parent words...
Hours: 0 | minutes: 1 | seconds: 59.201
...truncating output list...
(73218235, 2)
Hours: 0 | minutes: 1 | seconds: 59.3799


In [32]:
time_df = build_timing_and_output_objects(output_time_list, ls_df = ls_df)

In [ ]:
# yikes! That is slower!
# CPU: 31 seconds, from the Exp_01
# GPU: 119 seconds. 
# Not quite 90 seconds slower.

In [34]:
# count using numpy, and then create a Counter object
from_word_counter = cp.unique(ar=output_list[:, 1], return_counts=True)
to_word_counter = cp.unique(ar=output_list[:, 0], return_counts=True)


In [35]:
# zip with asterisk notation won't work on CuPy data types
from_word_counter = collections.Counter({wg_id: wg_count for wg_id, wg_count in zip(cp.asnumpy(from_word_counter[0]), cp.asnumpy(from_word_counter[1]))})
to_word_counter = collections.Counter({wg_id: wg_count for wg_id, wg_count in zip(cp.asnumpy(to_word_counter[0]), cp.asnumpy(to_word_counter[1]))})

In [36]:
# check to see if the word 'acanthology', with word_group_id 746 is in the counter
print(from_word_counter[746]) # 26
print(to_word_counter[746]) # 329

26
4


In [37]:
# the take away: using a GPU can be faster.
# But, it seems to only be faster when working on large matrices. 
# On smaller matrices, there isn't any speed up. In fact, there is slow down.